In [6]:
import re, os, errno
from tqdm import tqdm

## Nombres de boletines

### Boletines RAC

El formato del nombre d elos archivos era irregular en el tema de los espacios.
A veces las fechas tenían un espacio extra entre el guión anterior al año y este ("- 2018").

In [6]:
ruta_rel = os.path.join('..', 'Boletines', 'RAC', 'txt')
ruta_abs = os.path.abspath(ruta_rel)
# Quitamos los directorios
archivos = [
    f for f in os.listdir(ruta_abs) 
    if os.path.isfile(os.path.join(ruta_abs, f))
]

for filename in archivos:
    archivo_original=os.path.join(ruta_abs, filename)
#     print(filename)
    remplazo = re.sub(
        'COMUNICADO (?P<digit>[\d]+)', 
        'COMUNICADO_\g<digit>', 
        filename
    )
    remplazo = re.sub(' ', '', remplazo)
    if remplazo != filename:
        print(remplazo)
        archivo_remplazo=os.path.join(ruta_abs, remplazo)
        os.rename(archivo_original, archivo_remplazo)

### Boletines JAMK

El formato de fecha en el nombre de los boletines a veces marcaba 2019 cuando todos son del 2018.

In [3]:
ruta_rel = os.path.join('..', 'Boletines', 'JAMK', 'txt')
ruta_abs = os.path.abspath(ruta_rel)
# Quitamos los directorios
archivos = [
    f for f in os.listdir(ruta_abs) 
    if os.path.isfile(os.path.join(ruta_abs, f))
]

for filename in archivos:
    archivo_original=os.path.join(ruta_abs, filename)
#     print(filename)
    remplazo = re.sub(
        '^(?P<digit>[\d]{4})19', 
        '\g<digit>18', 
        filename
    )
    if remplazo != filename:
        print(remplazo)
        archivo_remplazo=os.path.join(ruta_abs, remplazo)
        os.rename(archivo_original, archivo_remplazo)

## Contenido boletines

In [34]:
def preprocesar(ruta_dir, archivos):
    try:
        ruta_folder = os.path.join(ruta_abs, 'preprocesados')
        os.makedirs(ruta_folder) # cre
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    remplazos = [
        # Elimino bigotes al final de archivos de AMLO
        (r'\u00aa(\u00aa)+', ''),
        # Elimino bigotes al final de archivos de JAMK
        (r'OO(O)+', ''),
        # Elimino bigotes al final de archivos de RAC
        (re.escape('**00**'), ''),
        (r'\*\*(\*)+', ''),
        # Remplaza cadenas encerradas dentro de doble guión bajo,
        # por ejemplo: _Hola_, _cómo estás_
        (r'_(?P<word>[^_]+)_', r'\g<word>'),
        # Remplazo caracter &nbsp (\u00A0) y tabuladores (\t) por espacio
        (r'[\u00A0\t]', ' '),
        # Remplazo repeticiones de más de dos espacios por espacio simple
        (r' ( )+', ' '),
        # Quito espacios al final e inicio de nuevas líneas
        (r'((( )+)\n)|(\n(( )+))', '\n'),
        # Remplazo saltos de líneas múltiples por un identificador especial
        (r'\n(\n)+', '<dnl>'),
        # Cambio saltos de linea simples por espacios
        (r'\n', ' '),
        # Remplazo identificador de saltos de líneas múltiple por linea simple
        (r'<dnl>', '\n'),
        # Elimino nuevas lineas al inicio y fin de archivo
        (r'(^( )*)|( )*$', ''),
        # Elimina links a imágenes tipo [C:\Hola\Mundo.jpg]
        (r'\[(.+)+\]', ''),
        # Elimino tablas al inicio
        (r'^\+.*\+\n', ''),
        # Quito los puntos que quedaron huéfanos en una sóla línea
        (r'^\.$', ''),
    ]
    for fname in tqdm(archivos):
        contenido = ""
        full_path = os.path.join(ruta_abs, fname)
#         print('Procesando: ', full_path, ' ...')
        with open(full_path, 'r', encoding='utf-8') as f:
            contenido = f.read()
        for viejo, nuevo in remplazos:
            contenido = re.sub(viejo, nuevo, contenido)
        with open(os.path.join(ruta_folder, fname), 'w', encoding='utf-8') as wf:
            wf.write(contenido)

In [35]:
for candidato in ['AMLO', 'RAC', 'JAMK']:
    ruta_rel = os.path.join('..', 'Boletines', candidato, 'txt')
    ruta_abs = os.path.abspath(ruta_rel)
    # Quitamos los directorios
    archivos = [
        f for f in os.listdir(ruta_abs) 
        if os.path.isfile(os.path.join(ruta_abs, f))
    ] 
#     print(ruta_abs)
    preprocesar(ruta_abs, archivos)

100%|███████████████████████████████████████████████████████████████████████████████| 145/145 [00:00<00:00, 764.61it/s]
